# WebBaseLoader

`WebBaseLoader` 는 웹 기반 문서를 로드하는 로더입니다. 

`bs4` 라이브러리를 사용하여 웹 페이지를 파싱합니다. 

- `bs4.SoupStrainer` 를 사용하여 파싱할 요소를 지정합니다. 
- `bs_kwargs` 매개변수를 사용하여 `bs4.SoupStrainer` 의 추가적인 인수를 지정합니다. 

**참고**

- [API 도큐먼트](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 뉴스기사 내용을 로드합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

SSL 인증 오류를 우회하기 위해, `"verify"` 옵션을 설정할 수 있습니다.

In [ ]:
# ssl 인증 우회
loader.requests_kwargs = {"verify": True}

# 데이터 로드
docs = loader.load()

여러 웹페이지를 한 번에 로드할 수도 있습니다. 이를 위해 **urls**의 리스트를 로더에 전달하면, 전달된 **urls**의 순서대로 문서 리스트를 반환합니다.

In [ ]:
loader = WebBaseLoader(
    web_paths=[
        "https://n.news.naver.com/article/437/0000378416",
        "https://n.news.naver.com/mnews/hotissue/article/092/0002340014?type=series&cid=2000063",
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# 데이터 로드
docs = loader.load()

# 문서 수 확인
print(len(docs))

웹에서 가져온 결과를 출력합니다.

In [ ]:
print(docs[0].page_content[:500])
print("===" * 10)
print(docs[1].page_content[:500])

여러 URL을 동시에 스크래핑하면 스크래핑 과정을 가속화할 수 있습니다.

동시 요청에는 합리적인 제한이 있으며, 기본값은 초당 2회입니다. 서버 부하에 대해 걱정하지 않거나, 스크래핑하는 서버를 제어하는 경우라면 `requests_per_second` 매개변수를 변경하여 최대 동시 요청 수를 늘릴 수 있습니다. 이 방법은 스크래핑 속도를 높일 수 있지만, 서버로부터 차단될 수 있으므로 주의해야 합니다.

In [ ]:
# jupyter notebook 에서만 실행(asyncio)
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# 초당 요청 수 설정
loader.requests_per_second = 1

# 비동기 로드
docs = loader.aload()

In [ ]:
# 결과 출력
docs

## 프록시 사용

IP 차단을 우회하기 위해 때때로 프록시를 사용할 필요가 있을 수 있습니다. 

프록시를 사용하려면 로더(및 그 아래의 `requests`)에 프록시 딕셔너리를 전달할 수 있습니다.

In [ ]:
loader = WebBaseLoader(
    "https://www.google.com/search?q=parrots",
    proxies={
        "http": "http://{username}:{password}:@proxy.service.com:6666/",
        "https": "https://{username}:{password}:@proxy.service.com:6666/",
    },
    # 웹 기반 로더 초기화
    # 프록시 설정
)

# 문서 로드
docs = loader.load()